# Version 4.0: 파인다이닝/스시오마카세 Source-based RAG 버전

이 노트북은 **원본 소스 데이터**를 활용한 RAG 방식입니다.

## 📌 핵심 차이점

### 기존 Example-based RAG (v3.0)
- 검색 대상: 과거 생성된 **요약문 예시**
- 목적: 요약문 **스타일** 학습
- 데이터 양: 적음 (요약문 180자)

### 새로운 Source-based RAG (v4.0)
- 검색 대상: 미쉐린/블루리본/인터뷰 등 **원본 소스**
- 목적: 풍부한 **배경 지식** 제공
- 데이터 양: 많음 (원본 텍스트 수백~수천자)

## 🗄️ 사용 컬렉션
- `fine_dining_sources` ← 원본 소스 검색 (NEW)
- `fine_dining_examples` ← 생성 성공 시 요약문 저장 (기존 유지)

## 🚀 사용 전 준비사항
1. `knowledge_base/source_indexer.ipynb`를 실행하여 원본 소스 인덱싱 필요
2. `fine_dining_sources` 컬렉션에 데이터가 있어야 함

## 예상 효과
- ✅ 더 풍부한 컨텍스트로 품질 향상
- ✅ Cold Start 문제 해결 (처음부터 원본 소스 활용 가능)
- ✅ 다양한 표현 가능 (예시 모방 탈피)

## ⚠️ 섹션 1-4: 최초 1회만 실행

In [ ]:
# 섹션 1: Package 설치
%pip install google-cloud-aiplatform google-genai python-dotenv chromadb

In [ ]:
# 섹션 2: 라이브러리 Import
import os
import json
import pandas as pd
import vertexai
from google import genai
from google.genai import types
from dotenv import load_dotenv
import chromadb
from datetime import datetime
from vertexai.language_models import TextEmbeddingModel

print("✅ 라이브러리 Import 완료")

In [ ]:
# 섹션 3: Vertex AI 초기화
load_dotenv()

PROJECT_ID = "wad-dw"
LOCATION = "us-central1"

vertexai.init(project=PROJECT_ID, location=LOCATION)
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

print(f"✅ Vertex AI 초기화 완료: {PROJECT_ID} ({LOCATION})")

In [ ]:
# 섹션 4: Chroma 벡터 DB 초기화 (공통 벡터 DB 사용)
try:
    # 공통 벡터 DB 경로: fine_dining_and_susi_omakase/../chroma_db = shop_summary/chroma_db
    chroma_client = chromadb.PersistentClient(path="../chroma_db")
    
    # 🆕 원본 소스 컬렉션 (검색용)
    fine_dining_sources = chroma_client.get_or_create_collection(
        name="fine_dining_sources",
        metadata={"hnsw:space": "cosine"}
    )
    
    # 기존 요약문 예시 컬렉션 (저장용)
    fine_dining_examples = chroma_client.get_or_create_collection(
        name="fine_dining_examples",
        metadata={"hnsw:space": "cosine"}
    )
    
    print("✅ Chroma 벡터 DB 초기화 완료 (공통 벡터 DB)")
    print(f"   - 저장 위치: ../chroma_db (shop_summary/chroma_db)")
    print(f"   - knowledge_base/source_indexer.ipynb에서 최초 1회 인덱싱 필요\n")
    print(f"   📂 원본 소스 (검색용):")
    print(f"      - fine_dining_sources: {fine_dining_sources.count()}개 청크")
    print(f"\n   📂 요약문 예시 (저장용):")
    print(f"      - fine_dining_examples: {fine_dining_examples.count()}개 요약문")
    
    if fine_dining_sources.count() == 0:
        print("\n   ⚠️ 경고: fine_dining_sources 컬렉션이 비어있습니다!")
        print("   knowledge_base/source_indexer.ipynb를 먼저 실행하여 원본 소스를 인덱싱하세요.")
    
except Exception as e:
    print(f"❌ Chroma 초기화 실패: {e}")
    chroma_client = None
    fine_dining_sources = None
    fine_dining_examples = None

## ✅ 섹션 5: RAG 함수 정의 (Source-based)

In [ ]:
# ========================================
# Source-based RAG 함수
# ========================================

def generate_embedding(text, model="text-embedding-004"):
    """텍스트를 768차원 벡터로 변환"""
    try:
        embedding_model = TextEmbeddingModel.from_pretrained(model)
        embeddings = embedding_model.get_embeddings([text])
        return embeddings[0].values
    except Exception as e:
        print(f"❌ 임베딩 생성 실패: {e}")
        return None


def retrieve_similar_sources(query_text, collection, top_k=3):
    """
    벡터 DB에서 유사한 원본 소스 검색
    
    주의: Example-based와 달리 top_k=3 권장 (더 많은 컨텍스트)
    """
    if not collection:
        print("⚠️ Chroma 연결 없음 - RAG 건너뛰기")
        return []
    
    if collection.count() == 0:
        print("⚠️ 원본 소스 컬렉션이 비어있습니다")
        return []
    
    try:
        # 쿼리 임베딩 생성
        query_embedding = generate_embedding(query_text)
        if not query_embedding:
            return []
        
        # Chroma 벡터 검색
        results = collection.query(
            query_embeddings=[query_embedding],
            n_results=min(top_k, collection.count()),
            include=["documents", "metadatas", "distances"]
        )
        
        # Chroma 결과 형식 변환
        formatted_results = []
        
        # results는 배치 형식: {'ids': [[...]], 'documents': [[...]], ...}
        if results['ids'] and len(results['ids'][0]) > 0:
            for i in range(len(results['ids'][0])):
                metadata = results['metadatas'][0][i]
                distance = results['distances'][0][i]
                document = results['documents'][0][i]
                
                # 코사인 거리를 유사도로 변환 (distance 범위: 0~2)
                # distance 0 = 동일, distance 2 = 정반대
                # similarity = 1 - (distance / 2)                
                similarity = 1.0 - (distance / 2.0)
                
                formatted_results.append({
                    "shop_name": metadata.get("shop_name"),
                    "source_type": metadata.get("source_type"),
                    "text": document,
                    "chunk_index": metadata.get("chunk_index", 0),
                    "score": similarity
                })
        
        return formatted_results
        
    except Exception as e:
        print(f"⚠️ 검색 실패: {e}")
        return []


def format_source_context(similar_sources):
    """
    검색된 원본 소스를 프롬프트용 컨텍스트로 포맷
    """
    if not similar_sources:
        return "[참고 가능한 원본 자료 없음]"
    
    context = "[참고 자료 - 원본 소스]\n\n"
    
    for idx, source in enumerate(similar_sources, 1):
        shop_name = source.get('shop_name', 'Unknown')
        source_type = source.get('source_type', 'unknown')
        text = source.get('text', '')
        score = source.get('score', 0)
        
        # 소스 타입 한글 변환
        type_mapping = {
            "michelin_review": "미쉐린 가이드",
            "blueribbon_review": "블루리본",
            "chef_interview": "셰프 인터뷰",
            "course_description": "코스 설명",
            "brand_philosophy": "브랜드 철학",
            "space_ambiance": "공간/분위기",
            "tradition_technique": "전통/기법"
        }
        type_kr = type_mapping.get(source_type, source_type)
        
        context += f"**참고 자료 {idx}: {shop_name} - {type_kr}**\n"
        context += f"{text}\n"
        context += f"\n유사도: {score:.3f}\n"
        context += "─" * 80 + "\n\n"
    
    return context


def store_successful_example(collection, shop_seq, shop_name, collected_info, 
                            title, summaries, category="fine_dining"):
    """
    성공한 요약문을 examples 컬렉션에 저장
    (기존 Example-based와 동일)
    """
    if not collection:
        print("⚠️ Chroma 연결 없음 - 저장 건너뛰기")
        return False
    
    try:
        info_embedding = generate_embedding(collected_info)
        
        if not info_embedding:
            print("❌ 임베딩 생성 실패")
            return False
        
        doc_id = f"{category}_{shop_seq}_{datetime.utcnow().strftime('%Y%m%d%H%M%S')}"
        
        collection.add(
            documents=[collected_info],
            embeddings=[info_embedding],
            metadatas=[{
                "shop_seq": shop_seq,
                "shop_name": shop_name,
                "category": category,
                "title": title,
                "summaries": json.dumps(summaries, ensure_ascii=False),
                "created_at": datetime.utcnow().isoformat()
            }],
            ids=[doc_id]
        )
        
        print(f"✅ 벡터 DB 저장 완료: {shop_name}")
        return True
        
    except Exception as e:
        print(f"❌ 저장 실패: {e}")
        return False


print("✅ Source-based RAG 함수 정의 완료")
print("   - generate_embedding(): 텍스트 → 768차원 벡터")
print("   - retrieve_similar_sources(): 원본 소스 검색 (NEW)")
print("   - format_source_context(): 원본 소스 포맷팅 (NEW)")
print("   - store_successful_example(): 요약문 저장 (기존)")

## ✅ 섹션 6: 매장 정보 입력

In [ ]:
# ========================================
# 매장 정보 입력 (다중 매장 모드)
# ========================================

# 1️⃣ 매장 기본 정보 (shop_seq, shop_name)
SHOPS = [
    (1, "모수"),
    (2, "밍글스"),
    (3, "가온"),
]

# 2️⃣ 매장별 수집된 정보 (매장명을 키로 사용)
COLLECTED_INFO = {
    "모수": "",
    "밍글스": "",
    "가온": "",
}

# 3️⃣ SHOP_LIST 자동 생성 (shop_seq, shop_name, collected_info 매핑)
SHOP_LIST = [
    {
        "shop_seq": seq,
        "shop_name": name,
        "collected_info": COLLECTED_INFO.get(name, "[정보 없음 - COLLECTED_INFO에 추가 필요]")
    }
    for seq, name in SHOPS
]

# 출력
print(f"📍 다중 매장 모드")
print(f"   - 총 매장 수: {len(SHOP_LIST)}개")
for idx, shop in enumerate(SHOP_LIST, 1):
    info_length = len(shop['collected_info'].strip())
    status = "✅" if info_length > 100 else "⚠️"
    print(f"   {status} {idx}. {shop['shop_name']} (seq: {shop['shop_seq']}) - {info_length:,}자")

## ✅ 섹션 7: 정보 수집

In [ ]:
# 이 섹션은 건너뜁니다.
# 섹션 6의 COLLECTED_INFO 딕셔너리에서 이미 정보를 입력했습니다.

print("✅ 다중 매장 모드")
print("   섹션 6의 COLLECTED_INFO에 매장별 정보를 입력하세요.")
print("   입력 완료 후 바로 섹션 8을 실행하세요.")

## ✅ 섹션 8: Source-based RAG 실행

In [ ]:
# ========================================
# Source-based RAG 프롬프트 실행
# ========================================

system_prompt = """
당신은 미쉐린 가이드와 블루리번 수준의 파인다이닝 레스토랑 전문 작가입니다.
제공된 원본 자료(미쉐린 가이드, 블루리본, 셰프 인터뷰 등)를 참고하여
매장의 정체성을 압축적으로 표현하는 요약문을 작성합니다.

## 작성 원칙
1. 사실 기반: 원본 자료의 정보만 사용
2. 구조화: 제목 1개 + 요약 문장 3개
3. 간결성: 각 문장 약 40-60자
4. 전문성: 미식 용어 정확히 사용
5. 객관성: 과장 금지
6. 창의성: 원본 자료를 재해석하여 새로운 표현으로 작성

## 요약 문장 구조
- **1번 문장**: 셰프 철학 / 브랜드 정체성 / 조리 철학
- **2번 문장**: 코스 구성 / 시그니처 메뉴 / 조리 기법
- **3번 문장**: 공간 특징 / 분위기 / 미식 경험

## 제목 작성 규칙
- 형식: "[공간 특징] 속, [시그니처 요소]를 [동사]하는 [장르]"
- 길이: 15-30자

## 출력 형식
{{
  "shop_seq": 숫자,
  "shop_name": "매장명",
  "title": "제목",
  "summaries": ["문장1", "문장2", "문장3"]
}}
"""

MODEL_NAME = "gemini-2.5-pro"
generation_config = types.GenerateContentConfig(
    temperature=0.5,
    max_output_tokens=4096,
    response_mime_type="application/json"
)

print(f"🤖 모델: {MODEL_NAME}")
print(f"   - Temperature: 0.5\n")


def generate_summary_with_source_rag(shop_seq, shop_name, collected_info):
    """
    Source-based RAG를 활용한 요약문 생성
    """
    
    # 1. 원본 소스에서 관련 정보 검색
    similar_sources = retrieve_similar_sources(
        query_text=collected_info,
        collection=fine_dining_sources,
        top_k=3  # Example-based보다 많이 (더 많은 컨텍스트)
    )
    
    # 2. 원본 소스 컨텍스트 생성
    source_context = format_source_context(similar_sources)
    
    # 3. 프롬프트 생성
    user_prompt = f"""
    {source_context}

    [생성할 매장 정보]
    매장명: {shop_name}
    Shop Seq: {shop_seq}

    [수집된 정보]
    {collected_info}

    위 참고 자료의 정보를 바탕으로 {shop_name} 매장의 요약문을 작성하세요.
    """
    
    try:
        response = client.models.generate_content(
            model=MODEL_NAME,
            contents=f"{system_prompt}\n\n{user_prompt}",
            config=generation_config
        )
        
        result_text = response.text
        result_json = json.loads(result_text)
        
        usage_metadata = response.usage_metadata if hasattr(response, 'usage_metadata') else None
        token_info = {}
        if usage_metadata:
            token_info = {
                "input_tokens": usage_metadata.prompt_token_count,
                "output_tokens": usage_metadata.candidates_token_count,
                "total_tokens": usage_metadata.total_token_count
            }
        
        return {
            "status": "success",
            "data": result_json,
            "tokens": token_info,
            "source_count": len(similar_sources)
        }
        
    except json.JSONDecodeError as e:
        return {
            "status": "error",
            "error_type": "JSONDecodeError",
            "message": str(e),
            "response": response.text if 'response' in locals() else None,
            "tokens": {},
            "source_count": len(similar_sources)
        }
    except Exception as e:
        return {
            "status": "error",
            "error_type": type(e).__name__,
            "message": str(e),
            "response": response.text if 'response' in locals() else None,
            "tokens": {},
            "source_count": len(similar_sources)
        }


# -----------------------------------------
# 다중 매장 처리
# -----------------------------------------
all_results = []
total_input_tokens = 0
total_output_tokens = 0
total_tokens = 0

print(f"📝 {len(SHOP_LIST)}개 매장 Source-based RAG 실행\n")
print("=" * 80)

for idx, shop in enumerate(SHOP_LIST, 1):
    shop_seq = shop["shop_seq"]
    shop_name = shop["shop_name"]
    collected_info = shop["collected_info"]
    
    print(f"\n[{idx}/{len(SHOP_LIST)}] {shop_name} 처리 중...")
    print(f"   🔍 원본 소스 검색 중...")
    
    result = generate_summary_with_source_rag(shop_seq, shop_name, collected_info)
    
    if result["status"] == "success":
        result_json = result["data"]
        all_results.append(result_json)
        
        # 토큰 사용량 출력 및 누적
        tokens = result.get("tokens", {})
        if tokens:
            input_tokens = tokens.get('input_tokens', 0)
            output_tokens = tokens.get('output_tokens', 0)
            total_tokens_shop = tokens.get('total_tokens', 0)
            
            total_input_tokens += input_tokens
            total_output_tokens += output_tokens
            total_tokens += total_tokens_shop
            
            print(f"   ✅ 성공 | 원본 소스: {result['source_count']}개 | Input: {input_tokens:,} | Output: {output_tokens:,} | Total: {total_tokens_shop:,} tokens")
        else:
            print(f"   ✅ 성공 | 원본 소스: {result['source_count']}개")
    else:
        print(f"   ❌ 실패: {result['error_type']} - {result['message']}")
        # 실패한 경우에도 기록
        all_results.append({
            "shop_seq": shop_seq,
            "shop_name": shop_name,
            "status": "error",
            "error": result['message']
        })

print("\n" + "=" * 80)
print(f"\n✅ 전체 완료: {len(all_results)}개 매장 처리")

# 성공/실패 개수
success_count = sum(1 for r in all_results if r.get("status") != "error")
error_count = len(all_results) - success_count
print(f"   - 성공: {success_count}개")
if error_count > 0:
    print(f"   - 실패: {error_count}개")

# 총 토큰 사용량 출력
if total_tokens > 0:
    print(f"\n📊 총 토큰 사용량:")
    print(f"   - Total Input:  {total_input_tokens:,} tokens")
    print(f"   - Total Output: {total_output_tokens:,} tokens")
    print(f"   - Total:        {total_tokens:,} tokens")

# -----------------------------------------
# 전체 결과 상세 출력
# -----------------------------------------
print("\n" + "=" * 80)
print("📋 전체 결과 상세보기")
print("=" * 80)

for idx, result in enumerate(all_results, 1):
    print(f"\n{'─' * 80}")
    print(f"[{idx}] {result.get('shop_name')}")
    print(f"{'─' * 80}")
    
    if result.get("status") != "error":
        # 성공한 경우 - 전체 내용 출력
        print(f"\n📌 제목:")
        print(f"   {result.get('title')}")
        
        print(f"\n📝 요약문:")
        summaries = result.get('summaries', [])
        for i, summary in enumerate(summaries, 1):
            print(f"   {i}. {summary}")
        
        print(f"\n✅ 상태: 성공")
    else:
        # 실패한 경우
        print(f"\n❌ 상태: 에러")
        print(f"   오류 내용: {result.get('error', 'N/A')}")

print("\n" + "=" * 80)

## ✅ 섹션 9-10: 검증 및 저장 (기존과 동일)

In [ ]:
# 섹션 9: 검증 (기존 코드와 동일)
# 섹션 10: 저장 (fine_dining_examples 컬렉션에 저장)

print("✅ 검증 및 저장 섹션")
print("   기존 main_rag.ipynb의 섹션 9-10과 동일하게 구현")
print("   (지면 관계상 생략 - 필요 시 복사)")

## 다음 단계

이제 Source-based RAG를 테스트했습니다!

### 다음: Hybrid RAG
- `main_rag_hybrid.ipynb`에서 **원본 소스 + 요약문 예시** 둘 다 활용
- 최상의 품질을 위한 하이브리드 접근